In [179]:
import numpy as np
from datetime import datetime as dt
import pandas as pd, datetime as dt

In [116]:
cd C:\Users\USER\OneDrive\Documents\GitHub\Datacracy_scala_hackathon\data

C:\Users\USER\OneDrive\Documents\GitHub\Datacracy_scala_hackathon\data


In [173]:
user = pd.read_csv('user_processed.csv',sep=',')
listening = pd.read_csv('listening_processed.csv',sep=',')
order = pd.read_csv('order_processed.csv',sep=',')
clustering = pd.read_csv('clustering_processed.csv',sep=',')


In [180]:
# change column name
order = order.rename(columns = {"OrderID (PK)": "oderID", "Transaction Date" : "transaction_date","UserID (FK)" : "userID", "Bill Type": "bill_type", "Total Amount": "total_amount" })
clustering = clustering.rename(columns = {"User_ID (FK)": "userID", "Type_user": "type_user", "MainCluster_ID" : "main_cluseringID", "MainCluster_Description" : "main_clustering_description"})
listening = listening.rename(columns = {"PlaylistID (PK)": "playlistID", "Playlist Name":"playlist_name", "Category":"cat", "Sub Category" :"sub_cat", "Playlist Type" :"palylist_type","Playlist Duration (min)": "playlist_duration","Actual Duration (min)":"actual_duation","UserID (FK)":"userID", "Listening Time":"listening_time", "Listening Date" :"listening_date", "Listening Datetime":"listening_datetime"})
user = user.rename(columns = {"UserID (PK)":"userID", "VIP hay Free":"user_type", "Gender":"gender", "Year of Birth":"year_of_birth","Age Range": "age_range", "Age":"age", "Region": "region","Operation Systems":"operation_system","Registration Date":"registration_date", "Total Listerning":"total_listening", "Total Listening Time (Min)":"total_listening_time",   })

# change date time
# xu li datetime
order.transaction_date = pd.to_datetime(order['transaction_date'], format='%Y-%m-%d')
user.registration_date = pd.to_datetime(user['registration_date'], format='%Y-%m-%d')
listening.listening_datetime = pd.to_datetime(listening['listening_datetime'], format='%Y-%m-%d')

# change date by weekly
def adding_by_week(df, date_field):
    # df["by_week"] = df[[date_field]].copy()
    # df.resample('W-Mon', on=date_field).sum().reset_index().sort_values(by=date_field)
    # order1.resample('W-Mon', on='transaction_date').sum().reset_index().sort_values(by='transaction_date')
    df['by_week'] = df.apply(lambda row: row[date_field] - dt.timedelta(days=row[date_field].weekday()), axis=1)
    df = df['by_week'].groupby(df['by_week']).count()

adding_by_week(order, "transaction_date")
adding_by_week(user, "registration_date")
adding_by_week(listening, "listening_datetime")

In [181]:
user.head()

,userID,user_type,gender,year_of_birth,age_range,age,region,operation_system,registration_date,total_listening,total_listening_time,by_week
0,86229,free,male,NaN,No information,NaN,VN,android,2020-11-27,218,4151.3,2020-11-23
1,48053,free,male,1995.0,25 - 34,26.0,VN,ios,2020-08-24,19,278.6,2020-08-24
2,46340,free,male,1988.0,25 - 34,33.0,VN,website,2020-08-19,28,170.5,2020-08-17
3,66659,free,male,1992.0,25 - 34,29.0,VN,website,2020-10-22,345,6721.4,2020-10-19
4,59396,free,male,NaN,No information,NaN,VN,android,2020-09-29,23,52.5,2020-09-28


### mục tiêu là mình làm xem nhóm nào đóng góp nhiều revenue nhất
1. paid user: dựa vào total amount
2. free user: dựa vào duration, số lượt nghe

In [183]:
order.head()

,oderID,transaction_date,userID,bill_type,total_amount,by_week
0,6706,2020-08-16,44822,VIP,199000,2020-08-10
1,6998,2020-08-23,47128,VIP,199000,2020-08-17
2,6927,2020-08-21,47134,VIP,199000,2020-08-17
3,7054,2020-08-25,45972,VIP,199000,2020-08-24
4,6929,2020-08-21,43556,VIP,199000,2020-08-17


In [188]:
total_amount = order.groupby('userID').sum()
sum_order = total_amount.total_amount.sum()
total_amount.head()

,oderID,total_amount
userID,,
40843,2544,27000
40861,7707,199000
41003,9599,599000
41014,15326,398000
41057,9313,599000


In [191]:
user_money = pd.merge(user,total_amount, how='left',left_on='userID', right_on='userID')
total = pd.merge(clustering,user_money, how='left',left_on='userID', right_on='userID')
total.head()

,userID,type_user,main_cluseringID,main_clustering_description,user_type,gender,year_of_birth,age_range,age,region,operation_system,registration_date,total_listening,total_listening_time,by_week,oderID,total_amount
0,40834,free,1,Hibernating,free,no information,NaN,No information,NaN,QT,ios,2020-07-01,2,8.7,2020-06-29,NaN,NaN
1,40854,free,1,Hibernating,free,female,1999.0,18 - 24,22.0,VN,website,2020-07-01,444,20988.0,2020-06-29,NaN,NaN
2,40876,free,1,Hibernating,free,no information,NaN,No information,NaN,VN,ios,2020-07-01,3,0.8,2020-06-29,NaN,NaN
3,40883,free,1,Hibernating,free,no information,NaN,No information,NaN,VN,ios,2020-07-01,5,2.9,2020-06-29,NaN,NaN
4,40891,free,1,Hibernating,free,no information,NaN,No information,NaN,VN,ios,2020-07-01,13,71.5,2020-06-29,NaN,NaN


### Paid contribution
1. by cluserter

In [197]:
df_paid_cluster = pd.DataFrame(total[total['type_user']=='paid'].groupby('main_clustering_description').total_amount.sum())
df_paid_cluster['%']=df_paid_cluster.total_amount/sum_order*100
df_paid_cluster.sort_values('%',ascending=False)

,total_amount,%
main_clustering_description,,
Loyal Users,262471000.0,36.638056
Potential Users,145295000.0,20.281579
New Paid Users,119967000.0,16.746070
Lost High-value,70042000.0,9.777090
Churn App Users,65260000.0,9.109576
Lost Users,50019000.0,6.982101
Need attention,3335000.0,0.465529


In [200]:
total_paid = total[total['type_user']=='paid']
total_paid.groupby([])

,userID,type_user,main_cluseringID,main_clustering_description,user_type,gender,year_of_birth,age_range,age,region,operation_system,registration_date,total_listening,total_listening_time,by_week,oderID,total_amount
24820,40843,paid,5,New Paid Users,free,male,1984.0,35 - 44,37.0,VN,ios,2020-07-01,144,1563.3,2020-06-29,2544.0,27000.0
24821,41955,paid,5,New Paid Users,free,male,NaN,No information,NaN,VN,android,2020-07-14,119,1707.1,2020-07-13,1379.0,45000.0
24822,44112,paid,5,New Paid Users,free,male,NaN,No information,NaN,VN,android,2020-08-12,217,1544.0,2020-08-10,1901.0,9000.0
24823,45239,paid,5,New Paid Users,free,male,NaN,No information,NaN,VN,android,2020-08-17,690,5670.1,2020-08-17,668.0,9000.0
24824,45647,paid,5,New Paid Users,free,no information,NaN,No information,NaN,VN,android,2020-08-18,40,236.6,2020-08-17,1264.0,45000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27453,76980,paid,3,Lost Users,free,no information,NaN,No information,NaN,VN,android,2020-11-10,22,12.7,2020-11-09,10131.0,199000.0
27454,78868,paid,3,Lost Users,free,female,1985.0,35 - 44,36.0,VN,ios,2020-11-13,159,3441.8,2020-11-09,10588.0,199000.0
27455,81001,paid,3,Lost Users,free,male,1986.0,35 - 44,35.0,QT,ios,2020-11-17,61,1441.5,2020-11-16,1790.0,298000.0
27456,83850,paid,3,Lost Users,free,female,1985.0,35 - 44,36.0,VN,ios,2020-11-22,75,842.6,2020-11-16,10703.0,159000.0


In [89]:
listening[listening['userID'] == 82846]

,playlistID,playlist_name,cat,sub_cat,palylist_type,playlist_duration,actual_duation,userID,listening_time,listening_date,listening_datetime,Unnamed: 11
0,4529,Dạy Tiếng Anh Cho Trẻ Mầm Non,Podcast,Học ngoại ngữ,free,44.9,0.1,82846,06:05:00,03-12-2020,2020-12-03 06:05:00,NaN
904,4563,Bí Kíp Nói Tiếng Anh Như Tiếng Việt,Podcast,Học ngoại ngữ,free,121.1,0.0,82846,21:46:00,18-11-2020,2020-11-18 21:46:00,NaN
3128,4563,Bí Kíp Nói Tiếng Anh Như Tiếng Việt,Podcast,Học ngoại ngữ,free,121.1,9.2,82846,11:20:00,29-12-2020,2020-12-29 11:20:00,NaN
6207,4563,Bí Kíp Nói Tiếng Anh Như Tiếng Việt,Podcast,Học ngoại ngữ,free,121.1,15.7,82846,21:46:00,25-11-2020,2020-11-25 21:46:00,NaN
7650,4529,Dạy Tiếng Anh Cho Trẻ Mầm Non,Podcast,Học ngoại ngữ,free,44.9,1.8,82846,22:13:00,01-12-2020,2020-12-01 22:13:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
653624,3872,[Sách Tóm Tắt] Warren Buffett - Quá Trình Hình...,Sách tóm tắt,Sách tóm tắt,free,26.3,12.4,82846,21:24:00,07-12-2020,2020-12-07 21:24:00,NaN
661291,4120,[Sách Tóm Tắt] Thế Giới Phẳng,Sách tóm tắt,Sách tóm tắt,free,30.6,8.2,82846,17:24:00,02-12-2020,2020-12-02 17:24:00,NaN
687352,4563,Bí Kíp Nói Tiếng Anh Như Tiếng Việt,Podcast,Học ngoại ngữ,free,121.1,16.9,82846,21:46:00,03-12-2020,2020-12-03 21:46:00,NaN
707299,4529,Dạy Tiếng Anh Cho Trẻ Mầm Non,Podcast,Học ngoại ngữ,free,44.9,0.9,82846,06:05:00,03-12-2020,2020-12-03 06:05:00,NaN


In [74]:
order

,oderID,transaction_date,userID,bill_type,total_amount
0,6706,2020-08-16,44822,VIP,199000
1,6998,2020-08-23,47128,VIP,199000
2,6927,2020-08-21,47134,VIP,199000
3,7054,2020-08-25,45972,VIP,199000
4,6929,2020-08-21,43556,VIP,199000
...,...,...,...,...,...
3570,13029,2020-12-31,106406,VIP,199000
3571,13027,2020-12-31,102442,VIP,599000
3572,13038,2020-12-31,98510,VIP,499000
3573,13032,2020-12-31,106480,VIP,159000


In [75]:
order.nunique()

oderID              3376
transaction_date     163
userID              2638
bill_type              3
total_amount          19
dtype: int64

In [76]:
set(order['bill_type'])

{'9K', 'VIP', 'XU'}

In [77]:
user_spending = order.groupby(['userID']).sum('total_amount')
user_spending

,oderID,total_amount
userID,,
40843,2544,27000
40861,7707,199000
41003,9599,599000
41014,15326,398000
41057,9313,599000
...,...,...
106773,12988,199000
106960,13012,159000
106971,13024,159000


In [78]:
clustering.head()

,userID,type_user,main_cluseringID,main_clustering_description
0,40834,free,1,Hibernating
1,40854,free,1,Hibernating
2,40876,free,1,Hibernating
3,40883,free,1,Hibernating
4,40891,free,1,Hibernating


In [79]:
clustering.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27458 entries, 0 to 27457
Data columns (total 4 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   userID                       27458 non-null  int64 
 1   type_user                    27458 non-null  object
 2   main_cluseringID             27458 non-null  int64 
 3   main_clustering_description  27458 non-null  object
dtypes: int64(2), object(2)
memory usage: 858.2+ KB


In [80]:
clustering.groupby(['type_user','main_clustering_description','main_cluseringID']).count()

userID
type_user main_clustering_description     main_cluseringID        
free      Defrost                         2                   1185
          Hibernating                     1                   3093
          Lost active user                4                   2510
          Loyal Users, Engaging Listeners 10                   427
          Loyal Users, Skimming Listeners 9                    342
          Need Attention                  3                   6196
          New User, Skimming Listeners    5                   4256
          New Users, Engaging Listeners   6                   4099
          Potential, Engaging Listeners   8                   1622
          Potential, Skimming Listeners   7                   1090
paid      Churn App Users                 1                    387
          Lost High-value                 4                    119
          Lost Users                      3                    254
          Loyal Users                     7                    552
          Need attention                  2                    175
          New Paid Users                  5                    424
          Potential Users                 6                    727

In [81]:
clustering.groupby('type_user').count()

,userID,main_cluseringID,main_clustering_description
type_user,,,
free,24820,24820,24820
paid,2638,2638,2638


### High values user:
1. Paid: chi trả > 199.000
2. Free: nghe tb tuần >=3

=> chọn từ clustering của nhóm


In [43]:
user_process = pd.merge(clustering,user, how='left',left_on='User_ID (FK)', right_on='UserID (PK)')
user_process

,User_ID (FK),Type_user,MainCluster_ID,MainCluster_Description,UserID (PK),VIP hay Free,Gender,Year of Birth,Age Range,Age,Region,Operation Systems,Registration Date,Total Listerning,Total Listening Time (Min)
0,40834,free,1,Hibernating,40834,free,no information,NaN,No information,NaN,QT,ios,2020-07-01,2,8.7
1,40854,free,1,Hibernating,40854,free,female,1999.0,18 - 24,22.0,VN,website,2020-07-01,444,20988.0
2,40876,free,1,Hibernating,40876,free,no information,NaN,No information,NaN,VN,ios,2020-07-01,3,0.8
3,40883,free,1,Hibernating,40883,free,no information,NaN,No information,NaN,VN,ios,2020-07-01,5,2.9
4,40891,free,1,Hibernating,40891,free,no information,NaN,No information,NaN,VN,ios,2020-07-01,13,71.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27453,76980,paid,3,Lost Users,76980,free,no information,NaN,No information,NaN,VN,android,2020-11-10,22,12.7
27454,78868,paid,3,Lost Users,78868,free,female,1985.0,35 - 44,36.0,VN,ios,2020-11-13,159,3441.8
27455,81001,paid,3,Lost Users,81001,free,male,1986.0,35 - 44,35.0,QT,ios,2020-11-17,61,1441.5
27456,83850,paid,3,Lost Users,83850,free,female,1985.0,35 - 44,36.0,VN,ios,2020-11-22,75,842.6


In [61]:
user_process_2 = pd.merge(user_process,user_spending, how='left',left_on='User_ID (FK)', right_on='UserID (FK)')
user_process_2

,User_ID (FK),Type_user,MainCluster_ID,MainCluster_Description,UserID (PK),VIP hay Free,Gender,Year of Birth,Age Range,Age,Region,Operation Systems,Registration Date,Total Listerning,Total Listening Time (Min),OrderID (PK),Total Amount
0,40834,free,1,Hibernating,40834,free,no information,NaN,No information,NaN,QT,ios,2020-07-01,2,8.7,NaN,NaN
1,40854,free,1,Hibernating,40854,free,female,1999.0,18 - 24,22.0,VN,website,2020-07-01,444,20988.0,NaN,NaN
2,40876,free,1,Hibernating,40876,free,no information,NaN,No information,NaN,VN,ios,2020-07-01,3,0.8,NaN,NaN
3,40883,free,1,Hibernating,40883,free,no information,NaN,No information,NaN,VN,ios,2020-07-01,5,2.9,NaN,NaN
4,40891,free,1,Hibernating,40891,free,no information,NaN,No information,NaN,VN,ios,2020-07-01,13,71.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27453,76980,paid,3,Lost Users,76980,free,no information,NaN,No information,NaN,VN,android,2020-11-10,22,12.7,10131.0,199000.0
27454,78868,paid,3,Lost Users,78868,free,female,1985.0,35 - 44,36.0,VN,ios,2020-11-13,159,3441.8,10588.0,199000.0
27455,81001,paid,3,Lost Users,81001,free,male,1986.0,35 - 44,35.0,QT,ios,2020-11-17,61,1441.5,1790.0,298000.0
27456,83850,paid,3,Lost Users,83850,free,female,1985.0,35 - 44,36.0,VN,ios,2020-11-22,75,842.6,10703.0,159000.0


In [84]:
# tao bảng user số lượng nghe, thời gian nghe
listening[listening.userID == '86229'].groupby('userID').count()

,playlistID,playlist_name,cat,sub_cat,palylist_type,playlist_duration,actual_duation,listening_time,listening_date,listening_datetime,Unnamed: 11
userID,,,,,,,,,,,
